Implemented hit selection, events scatter, ion heatmap, electrons time of flight, ions time of flight, merging multiple runs, spatial selection view, calibration choice of lines with corresponding zoomed-in views, calibration curve fit and apply with mq column

Additions:
+ make multiple threshold selections
+ spatial selection view with angle
+ fish plots
+ binning by number of events
+ choose appropriate binning for mq and ion tof
+ background with multiple runs downsampled to selected number of pulses
+ waterfall plots
+ heatmaps for binned number of events

# Imports and functions

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.optimize import curve_fit
from scipy.ndimage import gaussian_filter

In [ ]:
TIME_BETWEEN_PULSES = 3.54462e-6
CHANNELS_PER_PULSE = 14080


    
def read(runid):
    'Read the preprocessed data of run with ID runid saved in the h5 file with a corresponding name'
    'Outputs dataframes per event, per pulse, and xarrays etof, pnccd in that order'
    
    filename = '../preprocess/datarun' + str(runid) + '.h5'
    
    dfevent = pd.read_hdf(filename, 'dfevent')
    dfpulse = pd.read_hdf(filename, 'dfpulse')
    
    etof = xr.open_dataarray(filename, group="etof")
    pnccd = xr.open_dataarray(filename, group="pnccd")
    
    return dfevent, dfpulse, etof, pnccd



def events_selection(runs,thresholds,num_pulses):
    'Reads one or multiple runs from h5 files'
    'Makes a pulse selection based on the number of events per pulse between the defined thresholds'
    'If multiple runs are passed, will merge the runs, once hit selected'
    'Thresholds can be between one and three tuples (lower threshold, upper threshold)'
    'Downsamples by num_pulses'
    
    lower_threshold1, upper_threshold1 = thresholds[0]
    selected_dfevents1 = list()
    selected_dfpulses1 = list()
    selected_etofs1 = list()
    
    if len(thresholds) > 1:
        lower_threshold2, upper_threshold2 = thresholds[1]
        selected_dfevents2 = list()
        selected_dfpulses2 = list()
        selected_etofs2 = list()
    
    if len(thresholds) > 2:
        lower_threshold3, upper_threshold3 = thresholds[2]  
        selected_dfevents3 = list()
        selected_dfpulses3 = list()
        selected_etofs3 = list()
    
    dataframes = dict()
    
    if type(num_pulses) == int:
        num_pulses_run = int(num_pulses/len(runs))
    
    for run in runs:
        
        dfevent, dfpulse, etof, pnccd = read(run)
        
        selections = list()
        
        plt.figure()
        plt.scatter(dfpulse.pulseId,dfpulse.nevents_pulse,c='black',label='All pulses')
        
        if type(num_pulses) == int:
            dfpulse = dfpulse.sample(n=num_pulses_run)
            
        selected_dfpulse1 = dfpulse[lower_threshold1 < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold1]
        selected_dfevent1 = dfevent[dfevent.pulseId.isin(selected_dfpulse1.pulseId)]
        selected_etof1 = etof.sel(pulseId=etof.coords['pulseId'].isin(selected_dfpulse1.pulseId))
        selections.append((selected_dfevent1, selected_dfpulse1, selected_etof1))
        plt.scatter(selected_dfpulse1.pulseId,selected_dfpulse1.nevents_pulse,c='r',label=f'Between {lower_threshold1} and {upper_threshold1}')
        
        if len(thresholds) > 1:
            
            selected_dfpulse2 = dfpulse[lower_threshold2 < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold2]
            selected_dfevent2 = dfevent[dfevent.pulseId.isin(selected_dfpulse2.pulseId)]
            selected_etof2 = etof.sel(pulseId=etof.coords['pulseId'].isin(selected_dfpulse2.pulseId))
            selections.append((selected_dfevent2, selected_dfpulse2, selected_etof2))
            plt.scatter(selected_dfpulse2.pulseId,selected_dfpulse2.nevents_pulse,c='blue',label=f'Between {lower_threshold2} and {upper_threshold2}')
            
        if len(thresholds) > 2:
            
            selected_dfpulse3 = dfpulse[lower_threshold3 < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold3]
            selected_dfevent3 = dfevent[dfevent.pulseId.isin(selected_dfpulse3.pulseId)]
            selected_etof3 = etof.sel(pulseId=etof.coords['pulseId'].isin(selected_dfpulse3.pulseId))
            selections.append((selected_dfevent3, selected_dfpulse3, selected_etof3))
            plt.scatter(selected_dfpulse3.pulseId,selected_dfpulse3.nevents_pulse,c='g',label=f'Between {lower_threshold3} and {upper_threshold3}')  
        
        dataframes[run] = selections
          
        plt.xlabel('Pulse ID')
        plt.ylabel('Number of events per pulse')
        plt.legend()
        plt.title(f'Events per pulse with respect to pulse ID for run {run}')
        plt.show()

        
    for key, values in dataframes.items():
        
        selected_dfevents1.append(values[0][0])
        selected_dfpulses1.append(values[0][1])
        selected_etofs1.append(values[0][2])
        
        if len(thresholds) > 1:
            selected_dfevents2.append(values[1][0])
            selected_dfpulses2.append(values[1][1])
            selected_etofs2.append(values[1][2])
            
        if len(thresholds) > 2: 
            selected_dfevents3.append(values[2][0])
            selected_dfpulses3.append(values[2][1])
            selected_etofs3.append(values[2][2])
        
        
    merged_selection = list()
    
    merged_dfevent1 = pd.concat(selected_dfevents1)
    merged_dfevent1.reset_index(drop=True, inplace=True)
    
    merged_dfpulse1 = pd.concat(selected_dfpulses1)
    merged_dfpulse1.reset_index(drop=True, inplace=True)
    
    merged_etof1 = xr.concat(selected_etofs1, dim='pulseId')
    
    merged_selection.append((merged_dfevent1, merged_dfpulse1, merged_etof1))
    
    print(f"Number of pulses selected across {len(runs)} run(s) between {lower_threshold1} and {upper_threshold1} events: {len(merged_dfpulse1)}")
       
        
    if len(thresholds) > 1:
        merged_dfevent2 = pd.concat(selected_dfevents2)
        merged_dfevent2.reset_index(drop=True, inplace=True)
    
        merged_dfpulse2 = pd.concat(selected_dfpulses2)
        merged_dfpulse2.reset_index(drop=True, inplace=True)
    
        merged_etof2 = xr.concat(selected_etofs2, dim='pulseId')
        
        merged_selection.append((merged_dfevent2, merged_dfpulse2, merged_etof2))
        
        print(f"Number of pulses selected across {len(runs)} run(s) between {lower_threshold2} and {upper_threshold2} events: {len(merged_dfpulse2)}")
    
    
    if len(thresholds) > 2:
        merged_dfevent3 = pd.concat(selected_dfevents3)
        merged_dfevent3.reset_index(drop=True, inplace=True)
    
        merged_dfpulse3 = pd.concat(selected_dfpulses3)
        merged_dfpulse3.reset_index(drop=True, inplace=True)
    
        merged_etof3 = xr.concat(selected_etofs3, dim='pulseId')
        
        merged_selection.append((merged_dfevent3, merged_dfpulse3, merged_etof3))
        
        print(f"Number of pulses selected across {len(runs)} run(s) between {lower_threshold3} and {upper_threshold3} events: {len(merged_dfpulse3)}")
        
    
    return merged_selection



def heatmap(dfevent):
    'Creates heatmap of the ions hits, based on a dfevent dataframe'
    
    counts_df = dfevent.groupby(['x', 'y']).size().reset_index(name='count')
    heatmap_data = counts_df.pivot(index='y', columns='x', values='count')
    
    plt.figure()
    ax = sns.heatmap(heatmap_data, cmap='viridis',cbar_kws={'label': 'Number of events'})
    plt.title('Ion heatmap')
    plt.show()
    

    
def ion_tof(dfevent):
    'Plots ion time of flight data using dfevent dataframe'
    
    hist, bin_edges = np.histogram(dfevent.tof, bins=250000)
    hist1 = hist[:1500]
    bin_edges1 = bin_edges[:1501]
    
    plt.figure()
    plt.plot(bin_edges1[:-1], hist1)
    plt.xlabel('Time of flight (s)')
    plt.ylabel('Number of hits per bin')
    plt.title('Ions time of flight')
    plt.show()   
    
    
    
def e_tof(etof):
    'Plots electron time of flight data using etof xarray data'
    
    channel_time = TIME_BETWEEN_PULSES/CHANNELS_PER_PULSE
    
    xaxis = np.arange(14080)*channel_time
    avg_selected_etof = -np.mean(etof, axis=0)
    
    plt.figure()
    plt.plot(xaxis,avg_selected_etof/max(avg_selected_etof))
    plt.xlabel('Time of flight (s)')
    plt.ylabel('Normalized signal')
    plt.title('Electrons time of flight')
    plt.show()



def events_selection_plots(runs,thresholds,downsampling=None):
    'Runs functions events_selection, heatmap, e_tof, ion_tof'
    'Downsamples by downsampling integer value if one is given'
    
    selections = events_selection(runs,thresholds,downsampling)
    
    print(f'\n Plots for selection between {thresholds[0][0]} and {thresholds[0][1]} events:')
    selected_dfevent1, selected_dfpulse1, selected_etof1 = selections[0]
    heatmap(selected_dfevent1)
    ion_tof(selected_dfevent1)
    e_tof(selected_etof1)
    
    if len(selections) > 1:
        print(f'Plots for selection between {thresholds[1][0]} and {thresholds[1][1]} events:')
        selected_dfevent2, selected_dfpulse2, selected_etof2 = selections[1]
        heatmap(selected_dfevent2)
        ion_tof(selected_dfevent2)
        e_tof(selected_etof2)
    
    elif len(selections) > 2:
        print(f'Plots for selection between {thresholds[2][0]} and {thresholds[2][1]} events:')
        selected_dfevent3, selected_dfpulse3, selected_etof3 = selections[2]
        heatmap(selected_dfevent3)
        ion_tof(selected_dfevent3)
        e_tof(selected_etof3)
    
    return selections



def heatmap_with_zones(dfevent,zones):
    'Creates heatmap of the ions hits, based on a dfevent dataframe'
    'Draws a rectangle around zones defined by a list of tuples where each tuple represents a tilted zone (xstart, ystart, width, height, angle in degrees)'
    
    counts_df = dfevent.groupby(['x', 'y']).size().reset_index(name='count')
    heatmap_data = counts_df.pivot(index='y', columns='x', values='count')
    
    plt.figure()
    ax = sns.heatmap(heatmap_data, cmap='viridis', cbar_kws={'label': 'Number of events'})
    
    xlim = int(ax.get_xticklabels()[0].get_text())
    ylim = int(ax.get_yticklabels()[0].get_text())

    for zone in zones:
        x, y, width, height, angle = zone
        x_adjusted = x - xlim
        y_adjusted = y - ylim
        
        rect = plt.Rectangle((x_adjusted, y_adjusted), width, height, fill=False, edgecolor='red', lw=1, angle=angle)
        ax.add_patch(rect)
    
    plt.title('Ion heatmap')
    plt.show()
    
    
        
def spatial_ion_selection(dfevent,dfpulse,etof,zones):
    'Square selection from the heatmap using spatial coordinates'
    'Zone is a tuple representing a zone that is not tilted (xstart, ystart, width, height)'
    'Returns spatially selected dfevent,dfpulse,etof'
    
    heatmap_with_zones(dfevent,zones)
    
    selected_dfevents = []
    
    for zone in zones:
        
        xstart,ystart,width,height,angle = zone
    
        spatial_selected_dfevent = dfevent[dfevent.x > xstart][dfevent.x < xstart+width][dfevent.y > ystart][dfevent.y < ystart+height]
        selected_dfevents.append(spatial_selected_dfevent)
        
    merged_dfevent = pd.concat(selected_dfevents)
    merged_dfevent.reset_index(drop=True, inplace=True)
    
    spatial_selected_dfpulse = dfpulse[dfpulse.pulseId.isin(merged_dfevent.pulseId)]
    spatial_selected_etof = etof.sel(pulseId=etof.coords['pulseId'].isin(merged_dfevent.pulseId))
    
    return merged_dfevent,spatial_selected_dfpulse,spatial_selected_etof



def big_ion_tof(dfevent):
    'Plots widget of big ion time of flight data using dfevent dataframe'
    
    hist, bin_edges = np.histogram(dfevent.tof, bins=250000)
    hist1 = hist[:1500]
    bin_edges1 = bin_edges[:1501]
    
    plt.figure(figsize=(18, 8))
    plt.plot(bin_edges1[:-1], hist1, c='g')
    plt.xlabel('Time of flight (s)')
    plt.ylabel('Number of hits per bin')
    plt.title('Ions time of flight')
    plt.show()
    
 
    
def autoscale_y(ax,margin=0.1):
    """This function rescales the y-axis based on the data that is visible given the current xlim of the axis.
    ax -- a matplotlib axes object
    margin -- the fraction of the total height of the y-data to pad the upper and lower ylims"""

    import numpy as np

    def get_bottom_top(line):
        xd = line.get_xdata()
        yd = line.get_ydata()
        lo,hi = ax.get_xlim()
        y_displayed = yd[((xd>lo) & (xd<hi))]
        h = np.max(y_displayed) - np.min(y_displayed)
        bot = np.min(y_displayed)-margin*h
        top = np.max(y_displayed)+margin*h
        return bot,top

    lines = ax.get_lines()
    bot,top = np.inf, -np.inf

    for line in lines:
        new_bot, new_top = get_bottom_top(line)
        if new_bot < bot: bot = new_bot
        if new_top > top: top = new_top

    ax.set_ylim(bot,top)
    
    
    
def zoomed_ion_tof(dfevent,anchor):
    'Plots zoom around anchor point of ion time of flight data using dfevent dataframe'
    
    hist, bin_edges = np.histogram(dfevent.tof, bins=250000)
    hist1 = hist[:1500]
    bin_edges1 = bin_edges[:1501]
    
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(bin_edges1[:-1], hist1, c='g')
    plt.xlabel('Time of flight (s)')
    plt.ylabel('Number of hits per bin')
    plt.title('Ions time of flight')
    plt.xlim(anchor-1e-7,anchor+1e-7)
    autoscale_y(ax)
    ax.axvline(x=anchor, color='black', linestyle='--')
    plt.show()
    
    
    
def power_law(x, a, b):
    'Calibration fit power law'
    return a * x**b



def compute_calibration(calibration_lines):

    # Corresponding m/q argon values
    mq_lines = [40,20,40/3,40/4,40/5]
    
    # Initial guesses for parameters a and b
    initial_guess = [1.6e13, 2]

    # Perform the curve fitting
    params, covariance = curve_fit(power_law, calibration_lines, mq_lines, p0=initial_guess, maxfev=10000)

    # Extract the fitted values for a and b
    a_fit, b_fit = params

    print(f"The fit looks as follows: m/q = {a_fit:.2e} * tof^{b_fit:.2f}")
    
    return a_fit, b_fit



def calibrate(backgrd_dfevent):
    'Computes calibration by least mean squares using backgrd_dfevent'
    'Uses user input to compute fit based on displayed plots'
    
    # Show a large widget ion tof
    big_ion_tof(backgrd_dfevent)

    done = False
    while not done:

        # Ask for five numbers input
        anchors = []
        for i in range(5):
            value = input(f"Enter value Ar{i + 1}: ")
            try:
                anchors.append(float(value))
            except ValueError:
                print("Invalid input. Please enter a number.")

        # Show five additional plots based on inputs
        %matplotlib inline
        for anchor in anchors:
            zoomed_ion_tof(backgrd_dfevent,anchor)

        # Ask if the user is done
        done_response = input("Are you done? (y/n): ").strip().lower()
        if done_response == 'y':
            done = True
    
    # Compute calibration fit
    a_fit, b_fit = compute_calibration(anchors)
    
    return a_fit, b_fit



def apply_calibration(dfevents,a_fit,b_fit):
    'Applies calibration to each dfevent of the list of dfevents and outputs calibrated_dfevents list of dataframes with m/q column'
        
    calibrated_dfevents = list()
        
    for dfevent in dfevents:
        dfevent['mq'] = a_fit * dfevent.tof ** b_fit
        calibrated_dfevents.append(dfevent)
        
    return calibrated_dfevents



def mq_selection(calibrated_dfevent,dfpulse,etof,lower_mq,upper_mq):
    'Selects based on m/q values. Need to input calibrated_dfevent! Returns m/q selected dfevent,dfpulse,etof.'
    
    mqselected_dfevent = calibrated_dfevent[lower_mq < calibrated_dfevent.mq][calibrated_dfevent.mq < upper_mq]
    mqselected_dfpulse = dfpulse[dfpulse.pulseId.isin(mqselected_dfevent.pulseId)]
    mqselected_etof = etof.sel(pulseId=etof.coords['pulseId'].isin(mqselected_dfevent.pulseId))
    
    return mqselected_dfevent,mqselected_dfpulse,mqselected_etof



def find_rectangle_corners(zone):
    
    x, y, width, height, angle_degrees = zone
    
    angle_radians = np.deg2rad(angle_degrees)
    c, s = np.cos(angle_radians), np.sin(angle_radians)
    
    # Calculate the coordinates of the other three corners
    corners = np.array([
        [x, y],
        [x + width * c, y + width * s],
        [x + width * c - height * s, y + width * s + height * c],
        [x - height * s, y + height * c]
    ])
    
    return corners



def find_integer_coordinates(corners, zone):
    
    min_x, min_y = np.floor(np.min(corners, axis=0))
    max_x, max_y = np.ceil(np.max(corners, axis=0))

    integer_coordinates = []
    for x in range(int(min_x), int(max_x) + 1):
        for y in range(int(min_y), int(max_y) + 1):
            if is_inside(x, y, corners, zone):
                integer_coordinates.append((x, y))
    
    return integer_coordinates



def is_inside(x, y, corners, zone):
    
    ox, oy, width, height, angle_degrees = zone
    
    angle_radians = np.deg2rad(angle_degrees)
    c, s = np.cos(angle_radians), np.sin(angle_radians)
    
    rotated_x = (x-ox)*c + (y-oy)*s + ox
    rotated_y = -(x-ox)*s + (y-oy)*c + oy
    
    return ox <= rotated_x <= ox + width and oy <= rotated_y <= oy + height



def tilted_spatial_ion_selection(dfevent,dfpulse,etof,zones):
    'Selection from the heatmap using spatial coordinates'
    'Zones is a list of tuple representing zones - tilted or not - (xstart, ystart, width, height, angle in degrees)'
    'Returns spatially selected dfevent,dfpulse,etof'
    
    heatmap_with_zones(dfevent,zones)
    
    integer_coordinates = []
    for zone in zones:
        corners = find_rectangle_corners(zone)
        integer_coordinates.extend(find_integer_coordinates(corners, zone))
    
    x_coords, y_coords = zip(*integer_coordinates)
    
    spatial_selected_dfevent = dfevent[dfevent.x.isin(x_coords)][dfevent.y.isin(y_coords)]
    spatial_selected_dfpulse = dfpulse[dfpulse.pulseId.isin(spatial_selected_dfevent.pulseId)]
    spatial_selected_etof = etof.sel(pulseId=etof.coords['pulseId'].isin(spatial_selected_dfevent.pulseId))
    
    return spatial_selected_dfevent,spatial_selected_dfpulse,spatial_selected_etof



def fish_plot_ion_selection(dfevent,zone):
    'Selection from the heatmap using spatial coordinates'
    'Zone is a tuple representing a zone - tilted or not - (xstart, ystart, width, height, angle in degrees)'
    'Returns spatially selected dfevent'
    
    heatmap_with_zones(dfevent,[zone])
    
    corners = find_rectangle_corners(zone)
    integer_coordinates = find_integer_coordinates(corners, zone)
    
    x_coords, y_coords = zip(*integer_coordinates)
    
    spatial_selected_dfevent = dfevent[dfevent.x.isin(x_coords)][dfevent.y.isin(y_coords)]
    
    return spatial_selected_dfevent
    


def custom_format_tof(x, pos):
    return "{:.2g}".format(x)



def custom_format_mq(x, pos):
    return "{:.2f}".format(x)



def fish_plot_x(dfevent,zone):
    'Produces fish plots along x with respect to time of flight and m/q from dfevent dataframe and a zone defined as (startx, starty, width, height, angle in degrees)'
    
    fish_dfevent = fish_plot_ion_selection(dfevent,zone)
    
    tof_bins = 1000
    tof_bin_edges = np.linspace(0,TIME_BETWEEN_PULSES,tof_bins)
    
    fish_dfevent['tof_binned'] = pd.cut(fish_dfevent['tof'], bins=tof_bin_edges, labels=tof_bin_edges[:-1].astype('str'))
    tof_pivot_table = fish_dfevent.pivot_table(index='x', columns='tof_binned', aggfunc='size', fill_value=0)
    
    mq_bins = 500
    mq_bin_edges = np.linspace(0,200,mq_bins)
    
    fish_dfevent['mq_binned'] = pd.cut(fish_dfevent['mq'], bins=mq_bin_edges, labels=mq_bin_edges[:-1].astype('str'))
    mq_pivot_table = fish_dfevent.pivot_table(index='x', columns='mq_binned', aggfunc='size', fill_value=0)
    
    
    fig, axes = plt.subplots(2, 1, figsize=(30, 12))
    
    sns.heatmap(tof_pivot_table, cmap="viridis", ax=axes[0], norm=LogNorm())
    axes[0].set_title('Fish plot along x with respect to time of flight')
    #axes[0].xaxis.set_major_formatter(FuncFormatter(custom_format_tof))
    
    sns.heatmap(mq_pivot_table, cmap="viridis", ax=axes[1], norm=LogNorm())
    axes[1].set_title('Fish plot along x with respect to m/q')
    #axes[1].xaxis.set_major_formatter(FuncFormatter(custom_format_mq))

    plt.subplots_adjust(hspace=1)#0.5)
    plt.show()

    
    
def fish_plot_y(dfevent,zone):
    'Produces a fish plot along y from dfevent dataframe and a zone defined as (startx, starty, width, height, angle in degrees)'
    
    fish_dfevent = fish_plot_ion_selection(dfevent,zone)
    
    tof_bins = 1000
    tof_bin_edges = np.linspace(0,TIME_BETWEEN_PULSES,tof_bins)
    
    fish_dfevent['tof_binned'] = pd.cut(fish_dfevent['tof'], bins=tof_bin_edges, labels=tof_bin_edges[:-1].astype('str'))
    tof_pivot_table = fish_dfevent.pivot_table(index='y', columns='tof_binned', aggfunc='size', fill_value=0)
    
    mq_bins = 500
    mq_bin_edges = np.linspace(0,200,mq_bins)
    
    fish_dfevent['mq_binned'] = pd.cut(fish_dfevent['mq'], bins=mq_bin_edges, labels=mq_bin_edges[:-1].astype('str'))
    mq_pivot_table = fish_dfevent.pivot_table(index='y', columns='mq_binned', aggfunc='size', fill_value=0)
    
    
    fig, axes = plt.subplots(2, 1, figsize=(30, 12))
    
    sns.heatmap(tof_pivot_table, cmap="viridis", ax=axes[0], norm=LogNorm())
    axes[0].set_title('Fish plot along y with respect to time of flight')
    #axes[0].xaxis.set_major_formatter(FuncFormatter(custom_format_tof))
    
    sns.heatmap(mq_pivot_table, cmap="viridis", ax=axes[1], norm=LogNorm())
    axes[1].set_title('Fish plot along y with respect to m/q')
    #axes[1].xaxis.set_major_formatter(FuncFormatter(custom_format_mq))

    plt.subplots_adjust(hspace=1)#0.5)
    plt.show()